In [1]:
import pandas as pd
import numpy as np
import requests
from io import StringIO


In [2]:
inicio=20
valores=[]
while inicio != 24:
    
    fin=inicio+1
    valores.append(str(inicio)+str(fin))
    inicio +=1

In [3]:
def renombrarColumnas(tabla,columna1="HomeTeam",columna2="AwayTeam"):
    tabla[columna1] = tabla[columna1].replace({
        'Hercules': 'ID0031',
        'Levante': 'ID0024',
        'Malaga': 'ID0029',
        'Espanol': 'ID0021',
        'La Coruna': 'ID0028',
        'Mallorca': 'ID0015',
        'Osasuna': 'ID0012',
        'Santander': 'ID0032',
        'Sociedad': 'ID0006',
        'Ath Madrid': 'ID0003',
        'Ath Bilbao': 'ID0004',
        'Barcelona': 'ID0005',
        'Real Madrid': 'ID0002',
        'Valencia': 'ID0008',
        'Getafe': 'ID0010',
        'Sevilla': 'ID0016',
        'Sp Gijon': 'ID0030',
        'Villarreal': 'ID0014',
        'Zaragoza': 'ID0033',
        'Almeria': 'ID0020',
        'Granada': 'ID0019',
        'Betis': 'ID0007',
        'Vallecano': 'ID0013',
        'Celta': 'ID0017',
        'Valladolid': 'ID0022',
        'Elche': 'ID0023',
        'Eibar': 'ID0025',
        'Cordoba': 'ID0034',
        'Las Palmas': 'ID0009',
        'Leganes': 'ID0027',
        'Alaves': 'ID0011',
        'Girona': 'ID0001',
        'Huesca': 'ID0026',
        'Cadiz': 'ID0018'
    })
    
    tabla[columna2] = tabla[columna2].replace({
        'Hercules': 'ID0031',
        'Levante': 'ID0024',
        'Malaga': 'ID0029',
        'Espanol': 'ID0021',
        'La Coruna': 'ID0028',
        'Mallorca': 'ID0015',
        'Osasuna': 'ID0012',
        'Santander': 'ID0032',
        'Sociedad': 'ID0006',
        'Ath Madrid': 'ID0003',
        'Ath Bilbao': 'ID0004',
        'Barcelona': 'ID0005',
        'Real Madrid': 'ID0002',
        'Valencia': 'ID0008',
        'Getafe': 'ID0010',
        'Sevilla': 'ID0016',
        'Sp Gijon': 'ID0030',
        'Villarreal': 'ID0014',
        'Zaragoza': 'ID0033',
        'Almeria': 'ID0020',
        'Granada': 'ID0019',
        'Betis': 'ID0007',
        'Vallecano': 'ID0013',
        'Celta': 'ID0017',
        'Valladolid': 'ID0022',
        'Elche': 'ID0023',
        'Eibar': 'ID0025',
        'Cordoba': 'ID0034',
        'Las Palmas': 'ID0009',
        'Leganes': 'ID0027',
        'Alaves': 'ID0011',
        'Girona': 'ID0001',
        'Huesca': 'ID0026',
        'Cadiz': 'ID0018'
    })
    return tabla



In [4]:
def filtrarColumnas(data):
    if "Time" in data.columns:
        data = data.drop(["Time"], axis=1)
    data = data.drop(data.columns[range(tabla.columns.get_loc("B365H"),data.shape[1])], axis=1)
    
    if "B365H" in data.columns:
        data = data.drop("B365H",axis=1)
    return data

    

In [5]:
def crearMatriz(data):
    tmp_matrix = np.full((20, 40), np.nan)
    columnas =  [f'Jornada_{i}' for i in range(1, 41)]
    filas = sorted(data['HomeTeam'].unique())
    tmp_matrix = pd.DataFrame(tmp_matrix, columns=columnas)
    filas = filas
    tmp_matrix.index = filas
    tmp_matrix
    return tmp_matrix, filas

In [6]:
def goalDiff(data,filas,goal_diff):
    for j in filas:
        t=1
        #for i in range(data.shape[0] - 1, -1, -1):
        for i in range(data.shape[0]):
            if j == data.iloc[i, 2]:
                tmp_diff = data.iloc[i, 4] - data.iloc[i, 5]
                if pd.isna(goal_diff.loc[j, 'Jornada_1']):
                    goal_diff.loc[j, 'Jornada_1'] = tmp_diff
                    t+=1
                else:
                    goal_diff.loc[j, f'Jornada_{t}'] = goal_diff.loc[j, f'Jornada_{t-1}'] + tmp_diff
                    t += 1  
            
            elif j == data.iloc[i, 3]:
                tmp_diff = data.iloc[i, 5] - data.iloc[i, 4]
                if pd.isna(goal_diff.loc[j, 'Jornada_1']):
                    goal_diff.loc[j, 'Jornada_1'] = tmp_diff
                    t+=1
                else:
                    goal_diff.loc[j, f'Jornada_{t}'] = goal_diff.loc[j, f'Jornada_{t-1}'] + tmp_diff
                    t += 1
    
    
    goal_diff = goal_diff / goal_diff.max().max()
    goal_diff.insert(loc=0,column="Jornada_0",value=0)
    return goal_diff, filas

In [7]:
def cornersDiff(data,filas,corns_diff):
    for j in filas:
        t=1
        #for i in range(data.shape[0] - 1, -1, -1):
        for i in range(data.shape[0]):
            if j == data.iloc[i, 2]:
                tmp_diff = data.iloc[i, 16] - data.iloc[i, 17]
                if pd.isna(corns_diff.loc[j, 'Jornada_1']):
                    corns_diff.loc[j, 'Jornada_1'] = tmp_diff
                    t+=1
                else:
                    corns_diff.loc[j, f'Jornada_{t}'] = corns_diff.loc[j, f'Jornada_{t-1}'] + tmp_diff
                    t += 1  

            elif j == data.iloc[i, 3]:
                tmp_diff = data.iloc[i, 17] - data.iloc[i, 16]
                if pd.isna(corns_diff.loc[j, 'Jornada_1']):
                    corns_diff.loc[j, 'Jornada_1'] = tmp_diff
                    t+=1
                else:
                    corns_diff.loc[j, f'Jornada_{t}'] = corns_diff.loc[j, f'Jornada_{t-1}'] + tmp_diff
                    t += 1
                    
    corns_diff= corns_diff/ corns_diff.max().max()
    corns_diff.insert(loc=0,column="Jornada_0",value=0)
    return corns_diff, filas


In [8]:
def shootsDiff(data,filas,shoots_diff):
    for j in filas:
        t=1
        #for i in range(data.shape[0] - 1, -1, -1):
        for i in range(data.shape[0]):
            if j == data.iloc[i, 2]:
                tmp_diff = data.iloc[i, 12] - data.iloc[i, 13]
                if pd.isna(shoots_diff.loc[j, 'Jornada_1']):
                    shoots_diff.loc[j, 'Jornada_1'] = tmp_diff
                    t+=1
                else:
                    shoots_diff.loc[j, f'Jornada_{t}'] = shoots_diff.loc[j, f'Jornada_{t-1}'] + tmp_diff
                    t += 1  

            elif j == data.iloc[i, 3]:
                tmp_diff = data.iloc[i, 13] - data.iloc[i, 12]
                if pd.isna(shoots_diff.loc[j, 'Jornada_1']):
                    shoots_diff.loc[j, 'Jornada_1'] = tmp_diff
                    t+=1
                else:
                    shoots_diff.loc[j, f'Jornada_{t}'] = shoots_diff.loc[j, f'Jornada_{t-1}'] + tmp_diff
                    t += 1
                    
    shoots_diff= shoots_diff/ shoots_diff.max().max()
    shoots_diff.insert(loc=0,column="Jornada_0",value=0)
    return shoots_diff, filas    

In [9]:
def matrizVacia(data):
    cols = data.columns[[4, 5] + list(range(7, 22))]
    data=data.drop(cols,axis=1)
    data['HTCD'] = None
    data['ATCD'] = None
    data['HTGD'] = None
    data['ATGD'] = None
    data['HTSD'] = None
    data['ATSD'] = None
    data['HomeRating(ELO)'] = None
    data['AwayRating(ELO)'] = None
    return data

In [10]:
def resultadosCorner(data,filas,variable,resultados):
    for j in filas:
        t=0
        for i in range(data.shape[0]):
            if data.iloc[i,2] == j:
                resultados.loc[i,"HTCD"]=variable.loc[j,f'Jornada_{t}']
                t+=1
            elif data.iloc[i,3] == j:
                resultados.loc[i,"ATCD"]=variable.loc[j,f'Jornada_{t}']
                t+=1

    return resultados, filas                                           

In [11]:
def resultadosShoots(data,filas,variable,resultados):
    for j in filas:
        t=0
        for i in range(data.shape[0]):
            if data.iloc[i,2] == j:
                resultados.loc[i,"HTSD"]=variable.loc[j,f'Jornada_{t}']
                t+=1
            elif data.iloc[i,3] == j:
                resultados.loc[i,"ATSD"]=variable.loc[j,f'Jornada_{t}']
                t+=1

    return resultados, filas

In [12]:
def resultadosGoles(data,filas,variable,resultados):
    for j in filas:
        t=0
        for i in range(data.shape[0]):
            if data.iloc[i,2] == j:
                resultados.loc[i,"HTGD"]=variable.loc[j,f'Jornada_{t}']
                t+=1
            elif data.iloc[i,3] == j:
                resultados.loc[i,"ATGD"]=variable.loc[j,f'Jornada_{t}']
                t+=1

    return resultados, filas

In [13]:
def ratingsMatrix(data):
    tmp_matrix = np.full((20, 40), np.nan)
    columnas =  [f"{i}" for i in range(1, 41)]
    filas = sorted(data['HomeTeam'].unique())
    tmp_matrix = pd.DataFrame(tmp_matrix, columns=columnas)
    filas = filas
    tmp_matrix.index = filas
    tmp_matrix.insert(loc=0,column="0",value=1)
    return tmp_matrix, filas

In [14]:
def ratingDiff(data,rating_diff,gamma):
    for i in range(data.shape[0]):
        id1=data.iloc[i,2]
        id2=data.iloc[i,3]
        posit1=rating_diff.loc[id1,:].last_valid_index()
        posit2=rating_diff.loc[id2,:].last_valid_index()

        next_posit1= rating_diff.columns.get_loc(str(int(posit1)+1))
        next_posit2= rating_diff.columns.get_loc(str(int(posit2)+1))

        mu_alfa = rating_diff.loc[id1,str(posit1)]
        mu_beta = rating_diff.loc[id2,str(posit2)]

        if data.iloc[i,6] == "H":
            mu_alfa = mu_alfa + gamma * mu_beta
            mu_beta = mu_beta - gamma * mu_alfa

        elif data.iloc[i,6] == "A":
            mu_alfa = mu_alfa - gamma * mu_beta
            mu_beta = mu_beta + gamma * mu_alfa

        else:
            mu_alfa = mu_alfa - gamma* (mu_alfa - mu_beta)
            mu_beta = mu_beta - gamma* (mu_beta - mu_alfa)

        rating_diff.loc[id1,str(next_posit1)] = mu_alfa
        rating_diff.loc[id2,str(next_posit2)] = mu_beta
        
    columnas =  [f"Jornada_{i}" for i in range(0, 41)]
    rating_diff.columns = columnas
        
    return rating_diff

In [15]:
def resultadosRating(data,filas,variable,resultados):
    for j in filas:
        t=0
        for i in range(data.shape[0]):
            if data.iloc[i,2] == j:
                resultados.loc[i,'HomeRating(ELO)']=variable.loc[j,f'Jornada_{t}']
                t+=1
            elif data.iloc[i,3] == j:
                resultados.loc[i,'AwayRating(ELO)']=variable.loc[j,f'Jornada_{t}']
                t+=1

    return resultados, filas

In [16]:
final_results=pd.DataFrame()
for i in valores:
    #Obtener la tabla con la que vamos a trabajar:
    url = "https://www.football-data.co.uk/mmz4281/"+f"{i}"+"/SP1.csv"
    s = requests.get(url).content.decode('utf-8') 
    data=StringIO(s)
    tabla = pd.read_csv(data)
    
    #renombramos los nombres en "HomeTeam" "AwayTeam" por sus identificadores
    tabla = renombrarColumnas(tabla)
    tabla = filtrarColumnas(tabla)
    
    #Creamos las matrices temporales para las diferentes variables
    tmp_goal_diff, filas  = crearMatriz(tabla)
    tmp_corns_diff, filas = crearMatriz(tabla)
    tmp_shoots_diff, filas = crearMatriz(tabla)
    
    #Diferencia de goles
    goal_diff, filas =goalDiff(tabla,filas,tmp_goal_diff)
    corns_diff, filas = cornersDiff(tabla,filas,tmp_corns_diff)
    shoots_diff, filas = shootsDiff(tabla,filas,tmp_shoots_diff)

    #Generamos la matriz de resultados
    resultados=matrizVacia(tabla)
    
    #Añadimos los resultados a las columnas de corners
    resultados, filas= resultadosCorner(tabla,filas,corns_diff,resultados)
    resultados, filas= resultadosShoots(tabla,filas,shoots_diff,resultados)
    resultados, filas = resultadosGoles(tabla,filas,goal_diff, resultados)
    
    #Crear matrix de ratings (Sistema ELO)
    rating_diff, filas =ratingsMatrix(tabla)
    rating_diff = ratingDiff(tabla,rating_diff,gamma=0.05)
    
    resultados, filas = resultadosRating(tabla,filas,rating_diff,resultados)

    
    final_results= pd.concat([final_results,resultados])

In [17]:
pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None) 

In [18]:
final_results

,Div,Date,HomeTeam,AwayTeam,FTR,HTCD,ATCD,HTGD,ATGD,HTSD,ATSD,HomeRating(ELO),AwayRating(ELO)
0,SP1,12/09/2020,ID0025,ID0017,D,0,0,0,0,0,0,1,1
1,SP1,12/09/2020,ID0019,ID0004,H,0,0,0,0,0,0,1,1
2,SP1,12/09/2020,ID0018,ID0012,A,0,0,0,0,0,0,1,1
3,SP1,13/09/2020,ID0011,ID0007,A,0,0,0,0,0,0,1,1
4,SP1,13/09/2020,ID0022,ID0006,D,0,0,0,0,0,0,1,1
5,SP1,13/09/2020,ID0014,ID0026,D,0,0,0,0,0,0,1,1
6,SP1,13/09/2020,ID0008,ID0024,H,0,0,0,0,0,0,1,1
7,SP1,19/09/2020,ID0014,ID0025,H,0.0,0.0,0.0,0.0,0.008929,-0.017857,1.0,1.0
8,SP1,19/09/2020,ID0010,ID0012,H,0,-0.05,0,0.042553,0,0.008929,1,1.0475
9,SP1,19/09/2020,ID0017,ID0008,H,0.0,-0.0125,0.0,0.042553,0.017857,0.0,1.0,1.05


In [19]:
rating_diff

,Jornada_0,Jornada_1,Jornada_2,Jornada_3,Jornada_4,Jornada_5,Jornada_6,Jornada_7,Jornada_8,Jornada_9,Jornada_10,Jornada_11,Jornada_12,Jornada_13,Jornada_14,Jornada_15,Jornada_16,Jornada_17,Jornada_18,Jornada_19,Jornada_20,Jornada_21,Jornada_22,Jornada_23,Jornada_24,Jornada_25,Jornada_26,Jornada_27,Jornada_28,Jornada_29,Jornada_30,Jornada_31,Jornada_32,Jornada_33,Jornada_34,Jornada_35,Jornada_36,Jornada_37,Jornada_38,Jornada_39,Jornada_40
ID0001,1,1.0000,1.050000,1.092388,1.139775,1.181457,1.229331,1.273293,1.211889,1.255209,1.291281,1.328111,1.367450,1.417916,1.403027,1.449725,1.515638,1.552371,1.532202,1.597019,1.537768,1.568757,1.597936,1.570631,1.483795,1.417117,1.457359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ID0002,1,1.0475,1.092381,1.137269,1.187132,1.239239,1.183679,1.228070,1.288665,1.335256,1.313623,1.375830,1.360180,1.411216,1.446326,1.479283,1.461666,1.498866,1.527972,1.568488,1.589088,1.630696,1.675218,1.658186,1.736718,1.690124,1.723922,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ID0003,1,1.0500,1.049881,1.101869,1.049233,1.111195,1.155678,1.205040,1.261822,1.298652,1.341254,1.293401,1.334610,1.375247,1.305709,1.330932,1.269752,1.257406,1.296335,1.216484,1.239591,1.290452,1.334584,1.350764,1.317587,1.360620,1.310812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ID0004,1,0.9500,1.000000,1.052381,1.045398,1.097380,1.141919,1.131637,1.074855,1.113712,1.048221,1.044850,1.086059,1.120139,1.134284,1.185571,1.157052,1.223599,1.271823,1.304332,1.359135,1.308273,1.276087,1.309671,1.259363,1.333553,1.275105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ID0005,1,1.0000,1.052500,1.099750,1.149619,1.201969,1.246719,1.230059,1.276863,1.254119,1.309805,1.244123,1.296802,1.337586,1.321993,1.390756,1.318270,1.294231,1.316997,1.358749,1.412471,1.381461,1.423060,1.453849,1.402880,1.428052,1.474054,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ID0006,1,1.0000,0.997500,0.995012,1.042150,0.980188,1.029576,1.079063,1.135645,1.072554,1.118555,1.115769,1.053563,1.079586,1.124509,1.106978,1.144178,1.143317,1.123104,1.096054,1.028098,1.061199,1.051826,1.077766,1.041184,1.070861,1.036013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ID0007,1,1.0475,1.047625,0.995006,1.046994,0.986896,0.986789,0.977407,1.026894,1.018184,1.014260,1.059568,1.102962,1.092737,1.140186,1.108399,1.126944,1.127762,1.148993,1.113367,1.142370,1.074433,1.108017,1.097138,1.127681,1.102268,1.168945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ID0008,1,1.0475,1.097500,1.047500,0.997624,1.052718,1.041221,0.989742,0.938398,0.937479,0.980799,0.984001,1.020736,0.950175,0.933946,0.861460,0.812197,0.837500,0.880879,0.916058,0.949271,1.017228,0.952705,0.967652,0.924618,0.911564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ID0009,1,1.0000,0.945125,0.947744,0.890755,0.846141,0.887823,0.826420,0.868198,0.910148,0.859302,0.889998,0.957061,0.948972,0.891963,0.941426,0.978159,0.964492,0.900901,0.835051,0.867940,0.911626,0.832172,0.812286,0.860669,0.792638,0.793718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ID0010,1,1.0000,0.947500,0.997250,0.937893,0.987763,0.936284,0.946052,0.942711,0.935560,0.939691,0.936100,0.977181,0.963238,0.989261,0.942190,0.985263,1.022834,1.034563,0.991184,0.948861,0.971967,0.890432,0.900768,0.931776,0.920036,0.846333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
columnas

NameError: name 'columnas' is not defined